# Практика
Используемые библиотеки

In [1]:
import pandas as pd
import numpy as np
import requests
import io
import re


## Загрузка DataFrame
### Задача 1
На основании данных портала "Открытые данные России" о результатах Химического анализа родника в Нескучном саду https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad
средствами библиотеки __Pandas__ сформируйте поле выводов по каждому анализирумомому параметру.
Например, по показателю _pH_ получен результат _8.4 единицы pH_ при нормативе от _6 до 9 единиц pH_. Т.о. по данному показателю результат анализа в норме.
Для решения задачи необходимо программно "прочитать и понять" значение столбца "Норматив" и выделенное численное значение сравнить с нормативом согласно логике норматива. Например, __6 >= pH >= 9__.
В итоговом DataFrame столбец "Показатель" сделайте индексным.


Загзрузка DataFrame выполняется непосредственно c сайта "Открытые данные России" https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv (см. код ниже).


In [2]:
import pandas as pd
import numpy as np
import requests
import io
import re

In [3]:
url ="https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv"
s = requests.get(url).content
df=pd.read_csv(io.StringIO(s.decode('UTF8')))

In [4]:
#df=pd.read_csv("Химический анализ родника в Нескучном саду.csv", sep=';')
df

,Показатель,Единица измерений,Результат анализа,Норматив
0,pH,единицы pH,8.4,в пределах 6-9
1,Запах,баллы,1,не более 2-3
2,Цветность,градусы,б/цвета,не более 30
3,Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5"
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3"
6,Нитраты (по NO3),мг/дм3,24,не более 45
7,Фосфаты (P),мг/дм3,0.36,"не более 3,5"
8,Хлориды (Cl),мг/дм3,200,не более 350
9,Сульфаты (SO4),мг/дм3,189.5,не более 500


In [5]:
# Ваше решение

df['Результат анализа'] = pd.to_numeric(df['Результат анализа'], downcast='float', errors='coerce')
df['Норматив_условие'] = np.nan

for i in range(len(df)):
    s = df.Норматив[i].replace(',', '.') #меняем для преобразования во float
    s = re.findall(r"(?:\d*\.\d+|\d+)", s) #преобразование строки в список из элементов str под формат float
    s = [float(i) for i in s] #меняем формат со str на float, список состоит из 1 или 2 элементов
    #s = [float(''.join(filter(str.isdigit, x))) for x in s] #список в список из float

    if 'в пределах' in df.Норматив[i]:
        if s[0] <= df['Результат анализа'][i] <= s[1]:
            df.Норматив_условие[i] = 'норма'
        elif df['Результат анализа'][i] < s[0]:
            df.Норматив_условие[i] = 'ниже нормы'
        else:
            df.Норматив_условие[i] = 'выше нормы'

    else:
        if df['Результат анализа'][i] <= s[-1]:
            df.Норматив_условие[i] = 'норма'
        elif df['Результат анализа'][i] > s[-1]:
            df.Норматив_условие[i] = 'выше нормы'
        else:
            df.Норматив_условие[i] = 'не норма'

df



C:\Users\brigh\AppData\Local\Temp\ipykernel_11328\4218445962.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Норматив_условие[i] = 'норма'


,Показатель,Единица измерений,Результат анализа,Норматив,Норматив_условие
0,pH,единицы pH,8.400,в пределах 6-9,норма
1,Запах,баллы,1.000,не более 2-3,норма
2,Цветность,градусы,NaN,не более 30,не норма
3,Жёсткость,мг-эквл/дм3,9.200,в пределах 7-10,норма
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.420,"не более 1,5",норма
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3",норма
6,Нитраты (по NO3),мг/дм3,24.000,не более 45,норма
7,Фосфаты (P),мг/дм3,0.360,"не более 3,5",норма
8,Хлориды (Cl),мг/дм3,200.000,не более 350,норма
9,Сульфаты (SO4),мг/дм3,189.500,не более 500,норма


In [6]:
df.index = df.iloc[:,0] #df.set_index('Показатель') не работает, видимо из-за языка, с iloc тоже не прокатило
df = df.drop('Показатель', axis=1)
df.index.name = None
df.head()

,Единица измерений,Результат анализа,Норматив,Норматив_условие
pH,единицы pH,8.40,в пределах 6-9,норма
Запах,баллы,1.00,не более 2-3,норма
Цветность,градусы,NaN,не более 30,не норма
Жёсткость,мг-эквл/дм3,9.20,в пределах 7-10,норма
Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5",норма


## Теория вероятности. События

Требуется сгенерировать необходимые выборки и произвести по ним расчеты

### Задача 2
В ящике 5 апельсинов и 4 яблока. Наудачу выбираются 3 фрукта. Какова вероятность, что все три фрукта – апельсины?

В интернете полученный аналитически ответ 0.119. Подтверждается ли он эксперементально?


In [7]:
# Ваше решение
# Посчитаем вероятность 
P_apel = 5/9 * 4/8 * 3/7
P_apel

0.11904761904761905

In [8]:
%%time

p_apple_1 = 4/9
p_orange_1 = 5/9
conditions = ['апельсин', 'яблоко']
N = 100000

rez_test_1 = []
rez_test_2 = []
rez_test_3 = []

rez_test_1 = np.random.choice(conditions, N, p=[p_orange_1, p_apple_1])
for i in range(N):
    if rez_test_1[i] == conditions[0]:
        rez_test_2.append(np.random.choice(conditions, 1, p=[4/8, 4/8])[0])
        if rez_test_2[i] == conditions[0]:
            rez_test_3.append(np.random.choice(conditions, 1, p=[3/7, 4/7])[0])
        else:
            rez_test_3.append(np.random.choice(conditions, 1, p=[4/7, 3/7])[0])
    else:
        rez_test_2.append(np.random.choice(conditions, 1, p=[5/8, 3/8])[0])
        if rez_test_2[i] == conditions[0]:
            rez_test_3.append(np.random.choice(conditions, 1, p=[4/7, 3/7])[0])
        else:
            rez_test_3.append(np.random.choice(conditions, 1, p=[5/7, 2/7])[0])

rez_test_1[0:10]

CPU times: total: 11.5 s
Wall time: 13 s


array(['апельсин', 'апельсин', 'апельсин', 'яблоко', 'апельсин',
       'апельсин', 'апельсин', 'яблоко', 'яблоко', 'апельсин'],
      dtype='<U8')

In [9]:
df = pd.DataFrame(zip(rez_test_1, rez_test_2, rez_test_3), columns = ['rez_test_1', 'rez_test_2', 'rez_test_3'])
df.head()

,rez_test_1,rez_test_2,rez_test_3
0,апельсин,апельсин,апельсин
1,апельсин,яблоко,апельсин
2,апельсин,яблоко,яблоко
3,яблоко,яблоко,апельсин
4,апельсин,апельсин,яблоко


In [10]:
len(df[(df.rez_test_1 == 'апельсин') & (df.rez_test_2 == 'апельсин') & (df.rez_test_3 == 'апельсин')]) / len(df)

0.11934

### Вариант Lambda

In [11]:
%%time

rez_test_1 = np.random.choice(conditions, N, p=[p_orange_1, p_apple_1])

rez_test_2 = list(map(lambda x:
np.random.choice(conditions, 1, p=[4/8, 4/8])[0]
               if x == conditions[0] 
               else np.random.choice(conditions, 1, p=[5/8, 3/8])[0],
               rez_test_1))

rez_test_3 = list(map(lambda x, y:
np.random.choice(conditions, 1, p=[3/7, 4/7])[0] if x == conditions[0] and y == conditions[0]
            else (np.random.choice(conditions, 1, p=[4/7, 3/7])[0] if x == conditions[0] and y == conditions[1]
            else np.random.choice(conditions, 1, p=[4/7, 3/7])[0] if x == conditions[1] and y == conditions[0]
            else np.random.choice(conditions, 1, p=[5/7, 2/7])[0]),
               rez_test_1, rez_test_2))



CPU times: total: 8.86 s
Wall time: 8.87 s


In [12]:
df = pd.DataFrame(zip(rez_test_1, rez_test_2, rez_test_3), columns = ['rez_test_1', 'rez_test_2', 'rez_test_3'])
df.head()

,rez_test_1,rez_test_2,rez_test_3
0,апельсин,яблоко,яблоко
1,апельсин,апельсин,яблоко
2,апельсин,апельсин,апельсин
3,апельсин,апельсин,яблоко
4,апельсин,яблоко,яблоко


In [13]:
len(df[(df.rez_test_1 == 'апельсин') & (df.rez_test_2 == 'апельсин') & (df.rez_test_3 == 'апельсин')]) / len(df)

0.11851

### Вывод: Подтверждается, на выборке 100 000 погрешность 0,1 - 0,15%

### Задача 3
Мастер, имея 10 деталей, из которых 3 – нестандартных, проверяет детали одну за другой, пока ему не попадется стандартная. Какова вероятность, что он проверит ровно две детали?


В интернете полученный аналитически ответ 7/30 или 0.23333. Подтверждается ли он эксперементально?

In [14]:
# Ваше решение
P_urna = 3/10 * 7/9
P_urna

0.23333333333333334

In [15]:
%%time

p_1 = 3/10
p_2 = 2/9
conditions = ['стандарт', 'не стандарт']
N = 100000

rez_test_1 = []
rez_test_2 = []

rez_test_1 = np.random.choice(conditions, N, p=[1-p_1, p_1])
for i in range(N):
    if rez_test_1[i] == conditions[0]:
        rez_test_2.append(np.nan)
    else:
        rez_test_2.append(np.random.choice(conditions, 1, p=[1-p_2, p_2])[0])

rez_test_1[0:10]

CPU times: total: 1.45 s
Wall time: 1.44 s


array(['не стандарт', 'не стандарт', 'стандарт', 'стандарт', 'стандарт',
       'не стандарт', 'стандарт', 'не стандарт', 'не стандарт',
       'не стандарт'], dtype='<U11')

In [16]:
df = pd.DataFrame(zip(rez_test_1, rez_test_2), columns = ['rez_test_1', 'rez_test_2'])
df.head()

,rez_test_1,rez_test_2
0,не стандарт,стандарт
1,не стандарт,стандарт
2,стандарт,NaN
3,стандарт,NaN
4,стандарт,NaN


In [17]:
len(df[(df.rez_test_1 == 'не стандарт') & (df.rez_test_2 == 'стандарт')]) / N

0.23437

### Вариант Lambda

In [18]:
%%time

rez_test_1 = np.random.choice(conditions, N, p=[1-p_1, p_1])

rez_test_2 = list(map(lambda x:
np.random.choice(conditions, 1, p=[1-p_2, p_2])[0] if x==conditions[1]
else np.nan, rez_test_1))

df = pd.DataFrame(zip(rez_test_1, rez_test_2), columns = ['rez_test_1', 'rez_test_2'])
len(df[(df.rez_test_1 == 'не стандарт') & (df.rez_test_2 == 'стандарт')]) / N

CPU times: total: 1.58 s
Wall time: 1.56 s


0.23419